# Amazon 추천 시스템 - 데이터 분석 및 EDA

## 목표
- 데이터셋 기본 통계 파악
- User/Item interaction 분포 분석
- Rating 패턴 분석
- Cold-start 문제 규모 파악
- 그래프 구조 분석

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'AppleGothic'  # Mac용 한글 폰트
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

sns.set_style('whitegrid')
print('라이브러리 로딩 완료')

## 1. 데이터 로딩

In [ ]:
# 데이터 로드
df = pd.read_csv('../data/amazon_train.csv')

print(f'데이터 로딩 완료: {len(df):,} interactions')
print(f'컬럼: {list(df.columns)}')
print('\n첫 5개 행:')
df.head()

## 2. 기본 통계

In [ ]:
# 기본 정보
n_users = df['user'].nunique()
n_items = df['item'].nunique()
n_interactions = len(df)
sparsity = 1 - (n_interactions / (n_users * n_items))

print('=' * 50)
print('데이터셋 기본 정보')
print('=' * 50)
print(f'총 Interactions: {n_interactions:,}')
print(f'고유 사용자 수: {n_users:,}')
print(f'고유 아이템 수: {n_items:,}')
print(f'Sparsity: {sparsity:.6f} ({sparsity*100:.4f}%)')
print(f'Density: {(1-sparsity)*100:.6f}%')
print('=' * 50)

## 3. Rating 분포 분석

In [ ]:
# Rating 분포
rating_counts = df['rating'].value_counts().sort_index()
rating_pct = (rating_counts / len(df) * 100).round(2)

print('Rating 분포:')
for rating, count in rating_counts.items():
    pct = rating_pct[rating]
    print(f'  Rating {rating}: {count:,} ({pct}%)')

# Good rating (≥4) 비율
good_rating_count = df[df['rating'] >= 4].shape[0]
good_rating_pct = good_rating_count / len(df) * 100
print(f'\nGood Rating (≥4): {good_rating_count:,} ({good_rating_pct:.2f}%)')

In [ ]:
# Rating 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
axes[0].bar(rating_counts.index, rating_counts.values, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Rating', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Rating 분포', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Pie chart
colors = ['#ff9999', '#ffcc99', '#ffff99', '#99ccff', '#99ff99']
axes[1].pie(rating_counts.values, labels=rating_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90)
axes[1].set_title('Rating 비율', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 4. 사용자별 Interaction 분석

In [ ]:
# 사용자별 interaction 수
user_interactions = df.groupby('user').size()

print('사용자별 Interaction 통계:')
print(f'  평균: {user_interactions.mean():.2f}')
print(f'  중앙값: {user_interactions.median():.2f}')
print(f'  최소: {user_interactions.min()}')
print(f'  최대: {user_interactions.max()}')
print(f'  표준편차: {user_interactions.std():.2f}')

# Quantiles
quantiles = [0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
print('\nQuantiles:')
for q in quantiles:
    val = user_interactions.quantile(q)
    print(f'  {int(q*100)}%: {val:.0f}')

In [ ]:
# Cold-start 분석
cold_start_threshold = 10

bins = [0, 5, 10, 20, 50, 100, 200, float('inf')]
labels = ['1-5', '6-10', '11-20', '21-50', '51-100', '101-200', '200+']

user_bins = pd.cut(user_interactions, bins=bins, labels=labels, right=True)
user_bin_counts = user_bins.value_counts().sort_index()

print('\nInteraction 수별 사용자 분포:')
for label, count in user_bin_counts.items():
    pct = count / len(user_interactions) * 100
    print(f'  {label:10s}: {count:6,} 명 ({pct:5.2f}%)')

# Cold-start 사용자
cold_start_users = (user_interactions <= cold_start_threshold).sum()
cold_start_pct = cold_start_users / len(user_interactions) * 100
print(f'\nCold-start 사용자 (≤{cold_start_threshold}개): {cold_start_users:,} 명 ({cold_start_pct:.2f}%)')

In [ ]:
# 사용자 interaction 분포 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Histogram
axes[0, 0].hist(user_interactions, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Interactions per User', fontsize=11)
axes[0, 0].set_ylabel('Frequency', fontsize=11)
axes[0, 0].set_title('사용자별 Interaction 분포', fontsize=12, fontweight='bold')
axes[0, 0].axvline(user_interactions.mean(), color='red', linestyle='--', 
                    label=f'Mean: {user_interactions.mean():.1f}')
axes[0, 0].axvline(user_interactions.median(), color='green', linestyle='--', 
                    label=f'Median: {user_interactions.median():.1f}')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Log scale histogram
axes[0, 1].hist(user_interactions, bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Interactions per User', fontsize=11)
axes[0, 1].set_ylabel('Frequency (log scale)', fontsize=11)
axes[0, 1].set_title('사용자별 Interaction 분포 (Log Scale)', fontsize=12, fontweight='bold')
axes[0, 1].set_yscale('log')
axes[0, 1].grid(alpha=0.3)

# 3. Binned bar chart
axes[1, 0].bar(range(len(user_bin_counts)), user_bin_counts.values, 
               color='lightgreen', edgecolor='black')
axes[1, 0].set_xticks(range(len(user_bin_counts)))
axes[1, 0].set_xticklabels(user_bin_counts.index, rotation=45)
axes[1, 0].set_xlabel('Interaction Range', fontsize=11)
axes[1, 0].set_ylabel('Number of Users', fontsize=11)
axes[1, 0].set_title('Interaction 구간별 사용자 수', fontsize=12, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. CDF
sorted_interactions = np.sort(user_interactions)
cdf = np.arange(1, len(sorted_interactions) + 1) / len(sorted_interactions)
axes[1, 1].plot(sorted_interactions, cdf, color='purple', linewidth=2)
axes[1, 1].set_xlabel('Interactions per User', fontsize=11)
axes[1, 1].set_ylabel('Cumulative Probability', fontsize=11)
axes[1, 1].set_title('누적 분포 함수 (CDF)', fontsize=12, fontweight='bold')
axes[1, 1].axhline(0.5, color='red', linestyle='--', alpha=0.5, label='50% CDF')
axes[1, 1].axvline(cold_start_threshold, color='orange', linestyle='--', 
                    alpha=0.5, label=f'Cold-start threshold ({cold_start_threshold})')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 5. 아이템별 Interaction 분석

In [ ]:
# 아이템별 interaction 수
item_interactions = df.groupby('item').size()

print('아이템별 Interaction 통계:')
print(f'  평균: {item_interactions.mean():.2f}')
print(f'  중앙값: {item_interactions.median():.2f}')
print(f'  최소: {item_interactions.min()}')
print(f'  최대: {item_interactions.max()}')
print(f'  표준편차: {item_interactions.std():.2f}')

# Quantiles
print('\nQuantiles:')
for q in quantiles:
    val = item_interactions.quantile(q)
    print(f'  {int(q*100)}%: {val:.0f}')

# Cold-start 아이템
cold_item_threshold = 10
cold_items = (item_interactions <= cold_item_threshold).sum()
cold_items_pct = cold_items / len(item_interactions) * 100
print(f'\nCold-start 아이템 (≤{cold_item_threshold}개): {cold_items:,} 개 ({cold_items_pct:.2f}%)')

In [ ]:
# 아이템 interaction 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(item_interactions, bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Interactions per Item', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('아이템별 Interaction 분포', fontsize=12, fontweight='bold')
axes[0].axvline(item_interactions.mean(), color='red', linestyle='--', 
                label=f'Mean: {item_interactions.mean():.1f}')
axes[0].axvline(item_interactions.median(), color='green', linestyle='--', 
                label=f'Median: {item_interactions.median():.1f}')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Log-Log plot
sorted_items = item_interactions.sort_values(ascending=False).reset_index(drop=True)
axes[1].loglog(range(1, len(sorted_items) + 1), sorted_items.values, 
               color='purple', linewidth=2)
axes[1].set_xlabel('Item Rank (log scale)', fontsize=11)
axes[1].set_ylabel('Interactions (log scale)', fontsize=11)
axes[1].set_title('아이템 인기도 분포 (Long-tail)', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 추천 개수 계산 (평가 규칙)

In [ ]:
# 추천 가능 개수 계산 함수
def get_k_for_user(interaction_count, threshold=0.5, min_k=2, cold_start_threshold=10):
    """
    사용자별 추천 개수 계산
    
    Args:
        interaction_count: 사용자의 interaction 수
        threshold: 추천 비율 (default: 0.5 = 50%)
        min_k: cold-start 사용자 최소 추천 개수 (default: 2)
        cold_start_threshold: cold-start 기준 (default: 10)
    
    Returns:
        추천 개수
    """
    if interaction_count <= cold_start_threshold:
        return min_k
    else:
        return int(interaction_count * threshold)

# 각 사용자별 추천 개수 계산
user_k = user_interactions.apply(get_k_for_user)

print('추천 개수 통계:')
print(f'  평균: {user_k.mean():.2f}')
print(f'  중앙값: {user_k.median():.2f}')
print(f'  최소: {user_k.min()}')
print(f'  최대: {user_k.max()}')
print(f'  표준편차: {user_k.std():.2f}')

# k값 분포
k_distribution = user_k.value_counts().sort_index()
print('\nTop 10 추천 개수 분포:')
for k, count in k_distribution.head(10).items():
    pct = count / len(user_k) * 100
    print(f'  k={k:3d}: {count:6,} 명 ({pct:5.2f}%)')

In [ ]:
# 추천 개수 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(user_k, bins=30, color='gold', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('k (Recommendations per User)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('사용자별 추천 개수 (k) 분포', fontsize=12, fontweight='bold')
axes[0].axvline(user_k.mean(), color='red', linestyle='--', 
                label=f'Mean: {user_k.mean():.1f}')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Top k values bar chart
top_k = k_distribution.head(15)
axes[1].bar(range(len(top_k)), top_k.values, color='teal', edgecolor='black')
axes[1].set_xticks(range(len(top_k)))
axes[1].set_xticklabels(top_k.index, rotation=45)
axes[1].set_xlabel('k value', fontsize=11)
axes[1].set_ylabel('Number of Users', fontsize=11)
axes[1].set_title('Top 15 추천 개수별 사용자 수', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 그래프 구조 분석

In [ ]:
# 이분 그래프 (Bipartite Graph) 분석
print('이분 그래프 구조:')
print(f'  노드 수 (Users + Items): {n_users + n_items:,}')
print(f'    - Users: {n_users:,}')
print(f'    - Items: {n_items:,}')
print(f'  엣지 수 (Interactions): {n_interactions:,}')
print(f'  최대 가능 엣지 수: {n_users * n_items:,}')
print(f'  Sparsity: {sparsity*100:.4f}%')

# 평균 degree
avg_user_degree = user_interactions.mean()
avg_item_degree = item_interactions.mean()
print(f'\n평균 Degree:')
print(f'  User: {avg_user_degree:.2f}')
print(f'  Item: {avg_item_degree:.2f}')

## 8. 핵심 인사이트 요약

In [ ]:
print('='*70)
print(' '*20 + '핵심 인사이트 요약')
print('='*70)

print('\n1. 데이터 규모')
print(f'   - {n_interactions:,}개 interactions, {n_users:,}명 사용자, {n_items:,}개 아이템')
print(f'   - 극도로 희소한 그래프 (Sparsity: {sparsity*100:.4f}%)')

print('\n2. Rating 특성')
print(f'   - 긍정 평가(≥4) 비율: {good_rating_pct:.2f}%')
print(f'   - Rating 5가 전체의 {rating_pct[5]:.1f}%로 압도적')

print('\n3. Cold-start 문제')
print(f'   - Cold-start 사용자 (≤10개): {cold_start_pct:.2f}%')
print(f'   - Cold-start 아이템 (≤10개): {cold_items_pct:.2f}%')
print(f'   - 중앙값 interactions: 사용자 {user_interactions.median():.0f}, 아이템 {item_interactions.median():.0f}')

print('\n4. Long-tail 분포')
print(f'   - 소수의 인기 아이템에 interaction 집중')
print(f'   - 아이템 상위 1%: {item_interactions.quantile(0.99):.0f}개 이상 interactions')
print(f'   - 아이템 하위 25%: {item_interactions.quantile(0.25):.0f}개 이하 interactions')

print('\n5. 추천 시스템 설계 고려사항')
print(f'   - 평균 추천 개수: {user_k.mean():.2f}개 (사용자별로 다름)')
print(f'   - Cold-start 사용자는 무조건 2개 추천')
print(f'   - 일반 사용자는 interaction의 50%까지 추천 가능')

print('\n6. 모델 선택 방향')
print(f'   - 희소한 그래프에 강한 모델 필요 (LightGCN 유력)')
print(f'   - Cold-start 처리 전략 필수')
print(f'   - 메모리 효율적 구현 필요 (256K × 74K 그래프)')

print('='*70)

## 9. 다음 단계

1. ✅ 데이터 분석 및 EDA 완료
2. ⏭️ 데이터 전처리 (Train/Val/Test 분할)
3. ⏭️ GNN 모델 구현 및 비교
4. ⏭️ 학습 및 평가